# Main programme avec interaction avec l'utilisateur



L'idée de ce dossier est de modéliser un CP avec interaction d'un décision-maker.

In [1]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np
from Solver import *
from User import *
import time

In [2]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# Interaction with the solver

data = []

with open('./example.data', 'r') as f:    
    for line in f:
        data.append(line.strip().split())

n = int(data[0][0])
m = int(data[0][1])

T_machine = []
T_duration = []

for i in range(1,n+1):
    for j in range(0, 2*m, 2):
        T_machine.append(int(data[i][j]))
        T_duration.append(int(data[i][j+1]))

duration = np.zeros((n, m))

for i in range(n):
    for j in range(m):
        ind_machine = int(T_machine[i*m + j])
        duration[i][ind_machine] = T_duration[i*m + j]


# --------- Call Solver constructor in Solver.py

solver = Solver(data)

model = CpoModel() 

# --------- Create the model variables 

solver.create_variables(model, n, m, duration)


# ------------ Solve the model

print("\nSolving the model...")
msol, nb_solution, runtime = solver.solve(model, 4, n, m, T_machine)

# ------------ Display the result

# for sol in msol:
#     sol.write()
print("Model solved !")


# ---------------- Interaction with the user

print("\n--------Interaction with the user...---------")

print("\nCreating the user...")
user = User()
print("User created !")

print("\nClassing solutions...")	
list_indice, list_equal = user.classerSolutions(msol)
print("Solutions classed !")

print("\nCreating preferences...")
pref = user.getPreferences()
print("Preferences created !")

print("\nDisplaying preferences...")
# for sol in pref:
#     sol.write()

# print("\nPreferences : ", pref)
# print(list_indice)
# print(list_equal)
variables = solver.get_variables()
matrix = user.matrix_pref(n, m, variables[0] , True)

print("\nTesting preferences...")

if user.test_preferences(msol, list_indice):
    print("\tDeux solutions qui se suivent respectent bien l'ordre")
else:
    print("\tDeux solutions qui se suivent ne respectent pas l'ordre")
    
if user.test_differences_sol(matrix):
    print("\tLes solutions proposées lors d'une itération sont différentes de celles de l'itération précédente")
else:
    print("\tLes solutions proposées lors d'une itération ne sont pas différentes de celles de l'itération précédente")



--------Main program is loading...---------

Solver initialized !

Creating the model variables...
Model variables created !

Solving the model...

Adding precedence constraints to the solver...
Precedence constraints added !

Adding disjunctive constraints to the solver...
Disjunctive constraints added !
Model solved !

--------Interaction with the user...---------

Creating the user...
User created !

Classing solutions...
Solutions classed !

Creating preferences...
Preferences created !

Displaying preferences...
[<docplex.cp.solution.CpoSolveResult object at 0x00000251437E1840>, <docplex.cp.solution.CpoSolveResult object at 0x00000251437E1540>, <docplex.cp.solution.CpoSolveResult object at 0x00000251437E1040>, <docplex.cp.solution.CpoSolveResult object at 0x00000251437E1940>]
Matrice des start des préférences : 
(model: 3431330713, solve: Feasible, search: SearchOngoing, solution: (objs: None, gaps: None))
"T0-0" = intervalVar(size=3)
(model: 3431330713, solve: Feasible, search: 

In [3]:
###  -------------- Iteration of the solver with the preferences

# ----------------- Add the preferences to the model

tps_max = 100
tps = runtime
it_max = 5
it = 1
k = 1

criterion = (tps < tps_max) and (it < it_max) 

while criterion :

    it += 1
    
    solver = Solver(data)
    model = CpoModel() 

    # --------- Create the model variables 

    solver.create_variables(model, n, m, duration) 

    # stop = int(input("Do you want to continue ? True(1) or False(0)"))

    bb = integer_var(0,1)
    solver.add_variable(bb)
    bb=1

    list_variables = solver.get_variables()
    variables = list_variables[0]

    for sol in pref:
        b = integer_var(0,1)
        solver.add_variable(b)
        b=0

        for i in range(n):
            for j in range(m):

                var_sol = sol.get_all_var_solutions()
                var_sol = var_sol[i*m +j]
                #a = model.interval_var(start = sol[variables[i][j]].start, end= sol[variables[i][j]].end, size=int(duration[i][j]), name="a{}{}".format(i,j))
                #b =max(b,logical_or((model.start_of(a) != model.start_of(variables[i][j])), (model.end_of(a) != model.end_of(variables[i][j]))))
                
                b =max(b,logical_or(var_sol.get_start() != model.start_of(variables[i][j]), var_sol.get_end() != model.end_of(variables[i][j])))
        b = (b!=0)
        bb = bb * b
    solver.add_constraint(bb==1)
    

# ------------ Solve the model

    print("\nSolving the model...")

    msol, nb_solution, runtime = solver.solve(model, k, n, m, T_machine)

# ------------ Display the result

    for sol in msol:
        sol.write()
    print("Model solved !")


# ---------------- Interaction with the user

    print("\n--------Interaction with the user...---------")

    print("\nClassing solutions...")	
    list_indice, list_equal = user.classerSolutions(msol)
    print("Solutions classed !")

    print("\nCreating preferences...")
    pref = user.getPreferences()
    print("Preferences created !")

    print("\nDisplaying preferences...")
    # for sol in pref:
    #     sol.write()

    # print("\nPreferences : ", pref)
    # print(list_indice)
    # print(list_equal)
    # print(len(pref))

    # starts = user.start_pref(n, m, variables, True)

    matrix = user.matrix_pref(n, m, variables, True)

    print("\nTesting preferences...")
    
    if user.test_preferences(msol, list_indice):
        print("\tDeux solutions qui se suivent respectent bien l'ordre")
    else:
        print("\tDeux solutions qui se suivent ne respectent pas l'ordre")
        
    if user.test_differences_sol(matrix):
        print("\tLes solutions proposées lors d'une itération sont différentes de celles de l'itération précédente")
    else:
        print("\tLes solutions proposées lors d'une itération ne sont pas différentes de celles de l'itération précédente")

    tps += runtime

    criterion = (tps < tps_max) and (it < it_max)

    if (it == it_max):
        print("The user has reached the maximum number of iterations !")
    # elif not stop:
    #     print("The user has chosen his/her solution !")
    elif (tps >= tps_max):
        print("The user has reached the maximum time !")


print("\n--------End of the interaction with the user...---------")


Solver initialized !

Creating the model variables...
Model variables created !

Solving the model...

Adding precedence constraints to the solver...
Precedence constraints added !

Adding disjunctive constraints to the solver...
Disjunctive constraints added !
-------------------------------------------------------------------------------
Model constraints: 37, variables: integer: 0, interval: 36, sequence: 6
Solve status: Feasible
Search status: SearchOngoing, stop cause: SearchHasNotBeenStopped
Solve time: 0.04 sec
-------------------------------------------------------------------------------
Variables:
   T0-0 = IntervalVarValue(start=6, end=9, size=3)
   T0-1 = IntervalVarValue(start=16, end=22, size=6)
   T0-2 = IntervalVarValue(start=5, end=6, size=1)
   T0-3 = IntervalVarValue(start=22, end=29, size=7)
   T0-4 = IntervalVarValue(start=54, end=60, size=6)
   T0-5 = IntervalVarValue(start=29, end=32, size=3)
   T1-0 = IntervalVarValue(start=42, end=52, size=10)
   T1-1 = Interv

IndexError: list index out of range